In [ ]:
# Install the required packages
!pip install -q transformers datasets evaluate

from datasets import load_dataset, Dataset, Features, ClassLabel, Array3D
from transformers import ViTFeatureExtractor, ViTForImageClassification, TrainingArguments, Trainer
from transformers.modeling_outputs import SequenceClassifierOutput
import numpy as np
import pandas as pd
import torch
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import evaluate

# Labels for emotion categories
string_labels = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise', 'Neutral']

# Load ViT feature extractor and model
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

# Load the FER-2013 dataset
fer_df = pd.read_csv("AUG_train_data.csv")
fer_df.head()

# Function to prepare FER data for training
def prepare_fer_data(data):
    image_list = []
    image_labels = list(map(int, data['emotion']))

    for row in data.index:
        image = np.fromstring(data.loc[row, 'pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image = np.repeat(image[..., np.newaxis], 3, axis=2)  # Convert grayscale to RGB
        image_list.append(image.tolist())

    output_df = pd.DataFrame(list(zip(image_list, image_labels)), columns=['img', 'label'])
    return output_df

# Split dataset into train, validation, and test sets
train_data, temp_data = train_test_split(fer_df, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Prepare the datasets
fer_train_df = prepare_fer_data(train_data)
fer_val_df = prepare_fer_data(val_data)
fer_test_df = prepare_fer_data(test_data)

# Convert to Hugging Face dataset format
train_ds = Dataset.from_pandas(fer_train_df)
val_ds = Dataset.from_pandas(fer_val_df)
test_ds = Dataset.from_pandas(fer_test_df)

# Preprocess the images using the ViT feature extractor
def preprocess_images(examples):
    images = [np.array(image, dtype=np.uint8) for image in examples['img']]
    images = [np.moveaxis(image, source=-1, destination=0) for image in images]
    inputs = feature_extractor(images=images)
    examples['pixel_values'] = inputs['pixel_values']
    return examples

# Define dataset features
features = Features({
    'label': ClassLabel(names=string_labels),
    'img': Array3D(dtype="int64", shape=(3, 48, 48)),
    'pixel_values': Array3D(dtype="float32", shape=(3, 224, 224)),
})

# Apply preprocessing
train_ds = train_ds.map(preprocess_images, batched=True, batch_size=1, features=features)
val_ds = val_ds.map(preprocess_images, batched=True, features=features)
test_ds = test_ds.map(preprocess_images, batched=True, features=features)

# Save preprocessed datasets
with open('preprocessed_train_ds.pickle', 'wb') as handle:
    pickle.dump(train_ds, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Define model
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k', num_labels=len(string_labels))



Map:   0%|          | 0/6144 [00:00<?, ? examples/s]

Map:   0%|          | 0/1317 [00:00<?, ? examples/s]

Map:   0%|          | 0/1317 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir="./logs",
)

# Define accuracy metric
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model on the test set
outputs = trainer.predict(test_ds)
print(outputs.metrics)

# Confusion matrix
y_true = outputs.label_ids
y_pred = np.argmax(outputs.predictions, axis=1)
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=string_labels, yticklabels=string_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.159003,0.583144
2,1.290000,1.020320,0.640091
3,0.890600,0.998176,0.641610
4,0.635200,0.942327,0.668185
5,0.635200,0.951547,0.677297
6,0.388100,0.978930,0.678815


KeyboardInterrupt: 

In [ ]:

from datasets import load_dataset, Dataset, Features, ClassLabel, Array3D
from transformers import ViTFeatureExtractor, ViTForImageClassification, TrainingArguments, Trainer
from transformers.modeling_outputs import SequenceClassifierOutput
import numpy as np
import pandas as pd
import torch
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import evaluate

# Labels for emotion categories
string_labels = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise', 'Neutral']

# Load ViT feature extractor and model
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

# Load the FER-2013 dataset
fer_df = pd.read_csv("AUG_train_data.csv")
fer_df.head()

new_test_df = pd.read_csv("test_dataset.csv")  # Ensure this file has a 'pixels' column

# Check the structure of the CSV to ensure it has the correct 'pixels' column
print(new_test_df.head())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


     id                                             pixels
0  5001  80 81 77 69 66 59 70 89 112 132 140 142 144 14...
1  5002  226 226 226 217 203 189 97 149 193 193 199 200...
2  5003  98 112 43 41 46 47 67 37 27 37 65 32 39 29 41 ...
3  5004  35 38 29 25 21 29 35 32 41 49 34 68 123 136 11...
4  5005  4 1 5 19 14 15 21 50 73 73 61 62 72 76 66 55 6...


In [ ]:

# Function to prepare the new test data
# Preprocess the images using the ViT feature extractor
def preprocess_images(examples):
    images = [np.array(image, dtype=np.uint8) for image in examples['img']]
    images = [np.moveaxis(image, source=-1, destination=0) for image in images]
    inputs = feature_extractor(images=images)
    examples['pixel_values'] = inputs['pixel_values']
    return examples

def prepare_new_test_data(data):
    image_list = []

    for row in data.index:
        # Convert the pixel string to a NumPy array
        image = np.fromstring(data.loc[row, 'pixels'], dtype=int, sep=' ')
        # Reshape the array to 48x48
        image = np.reshape(image, (48, 48))
        # Convert the grayscale image to RGB by repeating the single channel
        image = np.repeat(image[..., np.newaxis], 3, axis=2)
        image_list.append(image)

    return image_list

# Prepare the new test data
new_test_images = prepare_new_test_data(new_test_df)

# Convert the images to Hugging Face dataset format
new_test_ds = Dataset.from_dict({'img': new_test_images})

# Preprocess the images using the same feature extractor
new_test_ds = new_test_ds.map(preprocess_images, batched=True)



Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [ ]:

# Predict the emotions for the new test dataset
predictions = trainer.predict(new_test_ds)

# Get the predicted labels (as indices)
predicted_labels = np.argmax(predictions.predictions, axis=1)

# Map the predicted labels back to emotion strings
predicted_emotions = [string_labels[label] for label in predicted_labels]

# Add the predicted emotions to the original dataframe
new_test_df['emotion'] = predicted_emotions

# Define the string labels
string_labels = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise', 'Neutral']

# Load the CSV file
df = new_test_df

# Map the 'predicted_emotion' column to their corresponding indices from 'string_labels'
df['predicted_emotion'] = df['predicted_emotion'].apply(lambda x: string_labels.index(x))

# Save the updated DataFrame to a new CSV file
df.to_csv('updated_file.csv', index=False)

print("Emotions mapped to indices and saved to 'updated_file.csv'")

# Save the predictions to a CSV file
new_test_df.to_csv("predicted_emotions.csv", index=False)

print("Predictions saved to 'predicted_emotions.csv'")